# Dynamic picklist simulation

### Complete code:
- Functions for relaxed and regular slot
- Picklist generator
- Simulation data generator


In [1]:
# functions

# -*- coding: utf-8 -*-
# Picklist generator function for regular period
def picklist_id_gen(data):
    # The input df1 has all observations pertaining to all slots <= active slot,
    #     the observations with putlist_id already created need to be excluded from the function
    
    # Loop variables
    global min_quantity
    global picklist_id
    loop_counter = 0
    quantity = 0
    volume = 0
    weight = 0
    min_quantity = min_quantity
    # Retain only those observations with picklist_id = 0 
    df1 = data[data['picklist_id'].isin([0])]
    
    # Order all reservations by location id - all reservations with no picklist id will be sorted together irrresp. of their slots
    df1 = df1.sort_values('storage_location_location_sequence', ascending = 1)
    df1 = df1.reset_index()# reset indices to be inline with the storage location

    # Get the next reservation's quantity, volume, weight
    df1['lead_product_volume'] = df1.product_volume.shift(-1) 
    df1['lead_product_detail_weight'] = df1.product_detail_weight.shift(-1) 
    df1['lead_storage_location_location_sequence'] = df1.storage_location_location_sequence.shift(-1)
    # df1['picklist_id'] = 0
    loop_counter = 0# loop counter will retain the number of items added into the picklist
    for index, row in df1.iterrows():
        if loop_counter == 0:
            # Re-initialize loop variables
            quantity = 1
            volume = row['product_volume']
            weight = row['product_detail_weight']
            distance_to_next = 0
        # Assign next item values to the loop  variables
        quantity = quantity + 1
        volume = volume + row['lead_product_volume']
        weight = weight + row['lead_product_detail_weight']
        if quantity > max_quantity:
            # the items in picklist have exceeded maximum allowable items for that vertical hence, create picklist
            df1.loc[index-loop_counter:index,['picklist_id']] = picklist_id
            loop_counter = 0
            picklist_id += 1
        elif (row['lead_storage_location_location_sequence'] - row['storage_location_location_sequence']) > max_distance or volume > max_volume or weight > max_weight:
            # permissible limits of distance, weight and volume are breached by picking next item
            if quantity - 1 < min_quantity:
                # picklist does not contain the minimum required quantity to be created hence, discard the picklist
                loop_counter = 0
                continue
            else:
                # create picklist
                df1.loc[index-loop_counter:index,['picklist_id']] = picklist_id
                loop_counter = 0
                picklist_id += 1
        else:
            # picklist has room for filling more items and addition of next item won't breach loop parameters
            loop_counter += 1
    return df1

# Picklist generator function for relaxed period
def picklist_id_gen_rel(data):
    # The input df1 has all observations pertaining to all slots <= active slot,
    #     the observations with putlist_id already created need to be excluded from the function
    
    # Loop variables
    global min_quantity
    global picklist_id
    loop_counter = 0
    quantity = 0
    volume = 0
    weight = 0
    min_quantity = min_quantity

    # Retain only those observations with picklist_id = 0 
    df1 = data[data['picklist_id'].isin([0])]
    
    # Order all reservations by location id - all reservations with no picklist id will be sorted together irrresp. of their slots
    df1 = df1.sort_values('storage_location_location_sequence', ascending = 1)
    df1 = df1.reset_index()# reset indices to be inline with the storage location

    # Get the next reservation's quantity, volume, weight
    df1['lead_product_volume'] = df1.product_volume.shift(-1) 
    df1['lead_product_detail_weight'] = df1.product_detail_weight.shift(-1) 
    df1['lead_storage_location_location_sequence'] = df1.storage_location_location_sequence.shift(-1)
    # df1['picklist_id'] = 0
    loop_counter = 0# loop counter will retain the number of items added into the picklist
    for index, row in df1.iterrows():
        if loop_counter == 0:
            # Re-initialize loop variables
            quantity = 1
            volume = row['product_volume']
            weight = row['product_detail_weight']
            distance_to_next = 0
        # Assign next item values to the loop  variables
        quantity = quantity + 1
        volume = volume + row['lead_product_volume']
        weight = weight + row['lead_product_detail_weight']
        if quantity > max_quantity:
            # the items in picklist have exceeded maximum allowable items for that vertical hence, create picklist
            df1.loc[index-loop_counter:index,['picklist_id']] = picklist_id
            loop_counter = 0
            picklist_id += 1
        elif (row['lead_storage_location_location_sequence'] - row['storage_location_location_sequence']) > max_distance or volume > max_volume or weight > max_weight:
            # permissible limits of distance, weight and volume are breached by picking next item, create picklist
            # here we are ignoring the fact that the items in the picklist could be lower than min. required items bcoz. it's relax time!
            df1.loc[index-loop_counter:index,['picklist_id']] = picklist_id
            loop_counter = 0
            picklist_id += 1
        else:
            # picklist has room for filling more items and addition of next item won't breach loop parameters
            loop_counter += 1
    return df1

# function to create picklists by vertical
def picklist_df_creator(data):
    global picklist_id
    global list_of_slots
    df_op = pd.DataFrame()
    #list_of_slots = data['slot_id'].unique().tolist()
    for i in list_of_slots:
        df1 = data[data['slot_id'].isin(list(range(0, i + 1)))]
        if i!=min(list_of_slots):df1 = df1[~df1['reservation_id'].isin(df_op['reservation_id'])]
        if i < 9000:

            #print("Running regular logic | slot id = ", i)
            df1 = picklist_id_gen(df1)
            if i == 0:
                df_op = df1[~df1['picklist_id'].isin([0])]
            else:
                df_op = df_op.append(df1[df1['picklist_id'].isin(range(1, 100000))])
        else:

            #print("Running relaxed logic | slot id = ", i)
            df1 = picklist_id_gen_rel(df1)
            df_op = df_op.append(df1[df1['picklist_id'].isin(range(1, 100000))])

            # Jugaad code begins here
            if i == max(list_of_slots):
                # for the last iteration of slot_id
                # apply a picklist_id = 0 filter on df1 and append the dataframe with the df_op
                df1 = df1[df1['picklist_id'].isin([0])]
                df1['picklist_id'] = picklist_id
                picklist_id += 1
                #or replace NaN with > 1200 distance
                df_op = df_op.append(df1)
            # Jugaad code ends here
    return df_op




"""
data=data_all[data_all['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 00:00:00']

final_output['fulfill_item_unit_dispatch_by_cutoff'].value_counts()

data_all['fulfill_item_unit_dispatch_by_cutoff'].value_counts()

data_1=data_all[data_all['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 13:00:00'].to_csv('1_raw.csv')


output_1=final_output[final_output['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 13:00:00'].to_csv('1_output.csv')



df_out.slot_id.value_counts()
a=data.slot_id.value_counts().reset_index()


print(x for x in range(0, 10 + 1))

list(range(0,10+1))

"""

"\ndata=data_all[data_all['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 00:00:00']\n\nfinal_output['fulfill_item_unit_dispatch_by_cutoff'].value_counts()\n\ndata_all['fulfill_item_unit_dispatch_by_cutoff'].value_counts()\n\ndata_1=data_all[data_all['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 13:00:00'].to_csv('1_raw.csv')\n\n\noutput_1=final_output[final_output['fulfill_item_unit_dispatch_by_cutoff']=='2018-09-08 13:00:00'].to_csv('1_output.csv')\n\n\n\ndf_out.slot_id.value_counts()\na=data.slot_id.value_counts().reset_index()\n\n\nprint(x for x in range(0, 10 + 1))\n\nlist(range(0,10+1))\n\n"

In [3]:
# Dynamic picklist-v3

# Read in required packages
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime


def run_simulation(data):
    global list_of_slots
    global picklist_id
    data['fulfill_item_unit_dispatch_by_cutoff']=pd.to_datetime(data['fulfill_item_unit_dispatch_by_cutoff'])
    #Identify lsit of dbds present in data
    dbd=list(data['fulfill_item_unit_dispatch_by_cutoff'].unique())
    dbd.sort()
    
    dbds=pd.DataFrame(dbd,columns=['dbd'])
    
    #create column with previous dbd
    dbds['prev_dbd']=dbds.shift(1)
    dbds['dbd']=pd.to_datetime(dbds['dbd'])
    dbds['prev_dbd']=pd.to_datetime(dbds['prev_dbd'])
    dbds.index.name='dbd_sequence'
    
    #remove dbd without any previous dbd
    dbds=dbds[~dbds['prev_dbd'].isnull()].reset_index()
    
    dbds['dbd_sequence']=dbds['dbd_sequence']*100000
    
    # Remove duplicates from data
    data.drop_duplicates(keep = 'first', inplace = True)
    
    # Remove all orders with more than one reservations
    data.sort_values('fulfill_item_unit_ship_together_preference_id', inplace = True)
    data.drop_duplicates(subset ="fulfill_item_unit_ship_together_preference_id",keep =False,inplace = True)
    
    # Split timestamp into Date and Time
    data['reservation_created_timestamp'] = pd.to_datetime(data['reservation_created_timestamp'])
    
    # Create a new column with volume of product
    data['fulfill_item_unit_dispatch_by_cutoff']=pd.to_datetime(data['fulfill_item_unit_dispatch_by_cutoff'])
    data['product_volume'] = data['product_detail_height']*data['product_detail_length']*data['product_detail_breadth']
    
    data_all=data.copy()
    # User inputs
    global max_distance
    global min_quantity
    global max_quantity
    global max_weight
    global max_volume
    global slot_size
    global slot_size_rel
    
    #cut_off_current  = "2018-09-08 13:00:00"#pd.to_datetime(input('Enter the current cut-off timestamp : '))
    #cut_off_previous = "2018-09-08 00:00:00"#pd.to_datetime(input('Enter the previous cut-off timestamp : '))
    
    final_output=pd.DataFrame()
    
    #iterating over different dbds in the data
    for index_dbd,row_dbd in dbds.iterrows():
        print('DBD started : {}'.format(row_dbd['dbd']))
        cut_off_current=row_dbd['dbd']
        cut_off_previous=row_dbd['prev_dbd']
        data=data_all[data_all['fulfill_item_unit_dispatch_by_cutoff']==cut_off_current]
        
        # Define slots : at a dbd level
        data['slot_id'] = data['reservation_created_timestamp'].apply(lambda x:0 if x < (pd.to_datetime(cut_off_previous) - pd.DateOffset(hours = 1.5)) 
                                                                      else (math.ceil((x - (pd.to_datetime(cut_off_previous) - pd.DateOffset(hours = 1.5))).seconds/(60*slot_size)) 
                                                                            if x < (pd.to_datetime(cut_off_current) - pd.DateOffset(hours = 1.5))
                                                                            else 9000 + math.ceil((x - (pd.to_datetime(cut_off_current) - pd.DateOffset(hours = 1.5))).seconds/(60*slot_size_rel))))
        
        # create a list of slots object - this removes the issue of a pickzone not having enough slot to run the whole loop 
        list_of_slots = data['slot_id'].unique().tolist()
        list_of_slots.append(1000000)
        list_of_slots.sort()
        # Initialise picklist_id variable
        
        picklist_id =1
        data['picklist_id'] = 0
        df_op = pd.DataFrame()
        df_out = pd.DataFrame()
        pickzones_completed = 0
        list_of_pickzones = data['picklist_picking_zone'].astype('str').unique().tolist()
        list_of_pickzones.sort()
        
        
        for pickzone in data['picklist_picking_zone'].unique():
            #print('Pickzone started : {}'.format(pickzone))
            df = picklist_df_creator(data[data['picklist_picking_zone'] == pickzone])
            df_out = df_out.append(df)
            pickzones_completed += 1
            #print('Pickzone completed : {} | pickzones completed : {}/{}'.format(pickzone, pickzones_completed, data['picklist_picking_zone'].nunique()))
    
        final_output=final_output.append(df_out)
        print('Simulation completed for DBD: {}'.format(row_dbd['dbd']))
    #Adding dbd to picklists to differentiate between picklist ids of different dbds
    final_output['dbd_proxy']=        final_output['fulfill_item_unit_dispatch_by_cutoff'].apply(lambda x:int(datetime.strftime(x,'%y%m%d%H%M%S0000')))
    final_output['picklist_id']=final_output['dbd_proxy']+final_output['picklist_id']
    del final_output['dbd_proxy'] 
    return final_output

In [ ]:
# simulation_evaluation

# -*- coding: utf-8 -*-
"""
Created on Wed Oct 31 18:57:12 2018

@author: adam.muhammad
"""
# Set working directory
os.chdir(r'C:\Analytics\FC\Dynamic Picklist')

# Data import and manipulations
data = pd.read_csv('dpl_fulldaydbd_zones_removed.csv')
df1=pd.DataFrame()


"""
sql code = select ob.reservation_id,
ob.picklist_display_id,
ob.reservation_created_timestamp,
ob.picklist_created_timestamp,
ob.picklist_picking_zone,
ob.fulfill_item_unit_dispatch_by_cutoff,
b.storage_location_id,
b.storage_location_label,
b.storage_location_location_sequence,
c.product_detail_height,
c.product_detail_length,
c.product_detail_breadth,
c.product_detail_weight,
c.product_detail_cms_vertical,
ob.fulfill_item_unit_lpht as lpht,
ob.fulfill_item_unit_ship_together_preference_id
from ekl.scp_warehouse__fc_outbound_breach_unit_l2_fact ob 
left join ekl.scp_warehouse__fc_reservation_l0_fact a 
on ob.fulfill_reference_id = a.reservation_fulfill_reference_id 
and reservation_inv_source_type != 'non_fki'
and reservation_outbound_type = 'customer_reservation'
left join ekl.scp_warehouse__fc_storage_location_dim b 
on a.picklist_item_suggested_location_key = b.fc_storage_location_dim_key
left join ekl.scp_warehouse__fc_product_detail_dim c 
on a.reservation_product_key = c.fc_product_detail_dim_key
where ob.reservation_status <> 'cancelled'
and ob.fiu_warehouse_id = 'blr_wfld'
and ob.fulfill_item_unit_dispatch_by_cutoff = '2018-09-08 16:30:00';

"""

max_distance = 1200#float(input('Enter the maximum inter item distance travel(feet?) : '))# 1200 "feet"?
min_quantity = 5#float(input('Enter the minimu quantity to be carried in a toat(# of items) : '))# 5
max_quantity = 25#float(input('Enter the maximum quantity to be carried in a toat(# of items) : '))# this needs to be looked up using product vertical : 25
max_weight = 12#float(input('Enter maximum permissible weight of items in toat(kgs) : '))# based on 25 item basket : 12kg
max_volume = 3500#float(input('Enter the toat volume(inch^3) : '))# based on 25 item basket : 3500 cubic inch
slot_size = 15#float(input('Enter the slot size for regular slots(in mins) : '))# 15mins
slot_size_rel = 5#float(input('Enter the slot size for relaxed slots(in mins) : '))# 5mins
setup_time = 1200
percentile = 95

output=run_simulation(data)

############Evaluation of simulation output##########

"""  Run this part to evaluate original parameters

df=pd.read_csv('dpl_fulldaydbd_zones_removed.csv')
df.sort_values('fulfill_item_unit_ship_together_preference_id', inplace = True)
df.drop_duplicates(subset ="fulfill_item_unit_ship_together_preference_id",keep =False,inplace = True)
df=df[~df['picklist_picking_zone'].isnull()]

try:
    del df['picklist_id']
except:
    True

names=list(df)
names[names.index('picklist_display_id')]='picklist_id'
df.columns=names


picklists_orig=df.loc[:,['storage_location_location_sequence','picklist_id']].sort_values('picklist_id').set_index('picklist_id')

picklists_orig['distance']=picklists_orig.groupby('picklist_id')['storage_location_location_sequence'].max()-picklists_orig.groupby('picklist_id')['storage_location_location_sequence'].min()

picklists_orig['picklist_size']=picklists_orig.groupby('picklist_id')['storage_location_location_sequence'].count()

picklists_orig=picklists_orig.reset_index()

picklists_orig=picklists_orig[~picklists_orig['picklist_id'].duplicated()]

picklists_orig.dropna(inplace=True)

#picklists_orig=picklists_orig[~(np.abs(picklists_orig.distance-picklists_orig.distance.mean()) > (3*picklists_orig.distance.std()))]

#picklists_orig=picklists_orig[picklists_orig['distance']>np.percentile(picklists_orig['distance'],85)]

gross_cost_orig=picklists_orig['distance'].sum()+setup_time*picklists_orig['picklist_id'].count()
number_of_picklists_orig=picklists_orig['picklist_id'].count()
picklist_size_distribution_orig=picklists_orig['picklist_size'].value_counts()

items_picked_orig=picklists_orig['picklist_size'].sum()
"""

df=output.copy()

picklists=df.loc[:,['storage_location_location_sequence','picklist_id','slot_id']].sort_values('picklist_id').set_index('picklist_id')

picklists['distance']=picklists.groupby('picklist_id')['storage_location_location_sequence'].max()-picklists.groupby('picklist_id')['storage_location_location_sequence'].min()

picklists['picklist_size']=picklists.groupby('picklist_id')['storage_location_location_sequence'].count()

picklists=picklists.reset_index()

picklists=picklists[~picklists['picklist_id'].duplicated()]

#picklists=picklists[picklists['distance']<np.percentile(picklists['distance'],90)]

gross_cost=picklists['distance'].sum()+setup_time*picklists['picklist_id'].count()
number_of_picklists=picklists['picklist_id'].count()
#picklist_size_distribution=picklists['picklist_size'].value_counts()
#items_picked=picklists['picklist_size'].sum()
number_of_picklists_reg=picklists[picklists['slot_id']<9000]['picklist_id'].count()
number_of_picklists_single_item=picklists[picklists['picklist_size']==1]['picklist_id'].count()
average_picklist_size=picklists[picklists['slot_id']<9000]['picklist_size'].mean()
average_picklist_volume=output[['product_volume','picklist_id']].groupby('picklist_id').agg('sum')['product_volume'].mean()
average_picklist_weight=output[['product_detail_weight','picklist_id']].groupby('picklist_id').agg('sum')['product_detail_weight'].mean()


number_of_picklists_single_item=picklists[picklists['picklist_size']<9000]['picklist_id'].count()


gross_cost_orig/gross_cost

a=pd.DataFrame([[gross_cost,number_of_picklists,number_of_picklists_reg,number_of_picklists_single_item,average_picklist_size,average_picklist_volume,average_picklist_weight,max_distance,min_quantity,max_quantity,max_weight,max_volume,slot_size,slot_size_rel,setup_time]])

a.columns=['gross_cost','no_picklists','no_picklists_reg','no_picklists_single_item','average_picklist_size','average_picklist_volume','average_picklist_weight','max_distance','min_quantity','max_quantity','max_weight','max_volume','slot_size','slot_size_rel','setup_time']


max_distance,min_quantity,max_quantity,max_weight,max_volume,slot_size,slot_size_rel,setup_time

df1=df1.append(a)